<p align="left">
<img width="50%" src="https://drive.google.com/uc?export=view&id=10y3NKbbk7yt7cZDMszMt04g6NquTEa4p" alt="Carbon Logo" />
</p>


# Carbon Simulator - Litepaper Examples

Those are the examples from the [Litepaper][lp] "Use Cases" section, presented very concisely. Please see the book `CarbonSim-Examples.ipynb` for a longer discussion, more technical explanations, and additional examples.

[lp]:https://carbondefi.xyz/r/litepaper

In [1]:
from carbon import CarbonSimulatorUI

## LP Example: Limit Order

**Bob creates a strategy where he buys ETH when the price goes to 1500 USDC. He is willing to spend 5000 USDC for this. He would like to buy at this specific price only.**

In [2]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")

In [3]:
Sim.add_strategy(tkn="ETH", amt_sell=0, psell_start=10000000, psell_end=10000000, amt_buy=10000, pbuy_start=1500, pbuy_end=1500)["orders"]

KeyError: 'orders'

The market is attempting to pass the following orders against it. The last one fails because there is not enough USDC left.

In [ ]:
for size in [2,1.4,3,0.5]:
    result = Sim.trader_sells("ETH", size)
result

In [ ]:
Sim.state()["trades"].query("aggr==True")

After those trades, Bob purchased 6.4 ETH. This should have cost him USDC 9600

In [ ]:
print(6.4*1500)
Sim.state()["orders"].query("y_unit=='ETH'")["y"]

We confirm that: Bob as 400 USDC left from the initial 10000

In [ ]:
print(10000-6.4*1500)
Sim.state()["orders"].query("y_unit=='USDC'")["y"]

## LP Example: "Buy-Low Sell-High" Strategy

**Alice creates a strategy where she buys ETH as the price goes between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this. She also wants to sell her acquired ETH when the price goes from 2000 to 2500 USDC.**

In [ ]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")

In [ ]:
Sim.add_strategy(tkn="ETH", amt_sell=0, psell_start=2000, psell_end=2500, amt_buy=5000, pbuy_start=1600, pbuy_end=1500)["orders"]

The market is attempting to pass the following orders against it. The last one fails because there is not enough USDC left.

In [ ]:
for size in [1,1,1,1]:
    result = Sim.trader_sells(tkn="ETH", amt=size, pair='ETHUSDC')
result

In [ ]:
Sim.state()["trades"].query("aggr==True")

After those trades, Alice has 3 ETH, and 341 USDC left.

In [ ]:
Sim.state()["orders"][['y', 'y_unit']]

Then the marker turns, and it buys ETH at the price the Alice is willing to sell it

In [ ]:
for size in [1,1,1,1]:
    result = Sim.trader_buys(tkn="ETH", amt=size, pair='ETHUSDC')
result

In [ ]:
Sim.state()["trades"].query("aggr==True").query("tkn1=='ETH'")

At the end, Alice has 7050 USDC, against a starting value of 5000.

In [ ]:
Sim.state()["orders"][['y', 'y_unit']]

## LP Example: "Average-In" Order

**Jane creates a strategy to buy ETH between 1500 and 1600 USDC. She is willing to spend 5000 USDC for this.**

Note: this is simply Alice's strategy without the sell part.

In [ ]:
Sim = CarbonSimulatorUI(pair="ETHUSDC")

In [ ]:
Sim.add_strategy(tkn="ETH", amt_sell=0, psell_start=1000000, psell_end=1000000, amt_buy=5000, pbuy_start=1600, pbuy_end=1500)["orders"]

The market is attempting to pass the following orders against it. The last one fails because there is not enough USDC left.

In [ ]:
for size in [1,1,1,340/1500,1]:
    result = Sim.trader_sells(tkn="ETH", amt=size, pair="ETHUSDC")
result

In [ ]:
Sim.state()["trades"].query("aggr==True")

After those trades, Jane has purchase 3.2 ETH, and she has 1.2 USDC left. She bought at an average price of 1549.

In [ ]:
o = Sim.state()["orders"][['y', 'y_unit']]
print((5000 - o.iloc[0]['y'])/ o.iloc[0]['y'])
o

## LP Example: Token Distribution Order

**Token project XYZ creates a strategy to sell 1 million units of its XYZ token in the price range of 0.50-2.00. The project would like to “back-load” the distribution process, such that half of the tokens are sold between 1.50-2.00. In total, the project wishes to receive a minimum of 1.37m in cash for its tokens.**

In [ ]:
Sim = CarbonSimulatorUI(pair="XYZUSDC")

In [ ]:
Sim.add_strategy(tkn="XYZ", amt_sell=500000, psell_start=0.5, psell_end=2, amt_buy=0, pbuy_start=0.001, pbuy_end=0.001, pair='XYZUSDC')
Sim.add_strategy(tkn="XYZ", amt_sell=500000, psell_start=1.5, psell_end=2, amt_buy=0, pbuy_start=0.001, pbuy_end=0.001, pair='XYZUSDC')
Sim.state()["orders"]

The market is now buying

In [ ]:
for size in [100]*11:
    result = Sim.trader_buys(tkn="XYZ", amt=size*999.999, pair='XYZUSDC')
result

We see that initially the price is rising fast, but as soon as both positions yield tokens (above 1.50) there is more capacity in any given price range

In [ ]:
Sim.state()["trades"].query("aggr==True")[["amt1", "tkn1", "nroutes", "price"]]

After those trades, the token project has sold most of its tokens (just 1 left across both positions), and has received USD 1,366,023

In [ ]:
o = Sim.state()["orders"][['y', 'y_unit']]
o.query("y_unit=='XYZ'").sum()

In [ ]:
o.query("y_unit=='USDC'").sum()